# Match POIs To Neighbourhoods
* read City of Toronto OpenData shape file with neighbourhood definitions


In [11]:
import pandas as pd
import altair as alt
alt.renderers.enable('notebook') # show and interact ith plots in jupyter notebook
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

In [12]:
import geopandas as gpd

In [13]:
def point_inside_polygon(lat,lng,poly):

    """Return True if the point described by x, y is inside of the polygon
    described by the list of points [(x0, y0), (x1, y1), ... (xn, yn)] in
    ``poly``
    Code from http://www.ariel.com.au/a/python-point-int-poly.html which
    in turn was adapted from C code found at
    http://local.wasp.uwa.edu.au/~pbourke/geometry/insidepoly/
    
    https://github.com/craigmbooth/chicago_neighborhood_finder
    
    poly: a list of lat/long coordinates that define the boundary of an area
    """
    n = len(poly)
    inside =False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if lat > min(p1y,p2y):
            if lat <= max(p1y,p2y):
                if lng <= max(p1x,p2x):
                    if p1y != p2y:
                        xinters = (lat-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or lng <= xinters:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside


In [14]:
df_neighbourhoods = gpd.read_file("../data/Neighbourhoods/NEIGHBORHOODS_WGS84.shp")


In [15]:
df_neighbourhoods.head()


,AREA_S_CD,AREA_NAME,geometry
0,097,Yonge-St.Clair (97),"POLYGON ((-79.39119482700001 43.681081124, -79..."
1,027,York University Heights (27),"POLYGON ((-79.505287916 43.759873494, -79.5048..."
2,038,Lansing-Westgate (38),"POLYGON ((-79.439984311 43.761557655, -79.4400..."
3,031,Yorkdale-Glen Park (31),"POLYGON ((-79.439687326 43.705609818, -79.4401..."
4,016,Stonegate-Queensway (16),"POLYGON ((-79.49262119700001 43.64743635, -79...."


In [16]:
# conver to list of lat, long coords
df_neighbourhoods['geomlist'] = df_neighbourhoods['geometry'].apply(lambda x: list(x.exterior.coords))

In [17]:
df_neighbourhoods.head()

,AREA_S_CD,AREA_NAME,geometry,geomlist
0,097,Yonge-St.Clair (97),"POLYGON ((-79.39119482700001 43.681081124, -79...","[(-79.391194827, 43.681081124), (-79.391405432..."
1,027,York University Heights (27),"POLYGON ((-79.505287916 43.759873494, -79.5048...","[(-79.505287916, 43.759873494), (-79.504882558..."
2,038,Lansing-Westgate (38),"POLYGON ((-79.439984311 43.761557655, -79.4400...","[(-79.439984311, 43.761557655), (-79.440036657..."
3,031,Yorkdale-Glen Park (31),"POLYGON ((-79.439687326 43.705609818, -79.4401...","[(-79.439687326, 43.705609818), (-79.440110286..."
4,016,Stonegate-Queensway (16),"POLYGON ((-79.49262119700001 43.64743635, -79....","[(-79.492621197, 43.64743635), (-79.492769811,..."


In [26]:
poly = df_neighbourhoods.loc[0,'geomlist']
#point inside area
lat=43.685052
lng=-79.396575
point_inside_polygon(lat, lng, poly)

True

In [27]:
poly = df_neighbourhoods.loc[0,'geomlist']
# point outside area
lat=43.698740
lng=-79.400233
point_inside_polygon(lat, lng, poly)

False

* read in the base dataframe with our Points of Interest

In [9]:
df_poi = pd.read_csv('../data/building_styles.csv', index_col=False)


In [20]:
def get_neighbourhood(row):
    for ix, area in df_neighbourhoods.iterrows():
        is_in_area=False
        if row['latitude'] and row['longitude']:
            is_in_area = point_inside_polygon(row['latitude'], row['longitude'], area['geomlist'])
            if is_in_area:
                #found area, exit
                return area['AREA_NAME']
    return ""

In [21]:
df_poi.head()

,poi_id,name,build_year,build_decade,build_year_clean,demolished_year,address,latitude,longitude,external_url,image_url,heritage_status,current_use,poi_type,poi_type_simple,source,details,style
0,1,43 Cross Street,unknown,NaN,NaN,NaN,"43 Cross Street Weston York, ON",43.705088,-79.525501,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Weston Heritage Conservation District,NaN,Detached house,Building,http://www.acotoronto.ca/,NaN,Arts and Crafts
1,10,J. H. McKinnon House,1888,1880.0,1888.0,NaN,"506 Jarvis Street Church-Wellesley Toronto, ON",43.667350,-79.379120,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Heritage property,NaN,Detached house,Building,http://www.acotoronto.ca/,NaN,Queen Anne
2,16,121 Major Street,1885,1880.0,1885.0,NaN,"121 Major Street Harbord Village Toronto, ON",43.660810,-79.403780,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,NaN,NaN,Rowhouse,Building,http://www.acotoronto.ca/,NaN,Toronto Bay and Gable
3,45,22 Chestnut Park,1905,1900.0,1905.0,NaN,"22 Chestnut Park Rosedale Toronto, ON",43.679380,-79.388100,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,South Rosedale Heritage Conservation District,NaN,Detached house,Building,http://www.acotoronto.ca/,"First Occupant: Falconbridge, John D.\r\r\r\r\...",American colonial
4,60,23-25 Gifford Street,unknown,NaN,NaN,NaN,"23-25 Gifford Street Cabbagetown Toronto, ON",43.663470,-79.363190,http://www.acotoronto.ca/show_building.php?Bui...,http://www.acotoronto.ca/tobuilt_bk/php/Buildi...,Cabbagetown South Heritage Conservation District,NaN,Semi-detached house,Building,http://www.acotoronto.ca/,NaN,Arts and Crafts


* get the neighbourhood for each poi

In [22]:
df_poi['neighbourhood'] = df_poi.apply(lambda row: get_neighbourhood(row), axis=1)